# Setup Instructions

First, we will set up for running Selenium in Google Colab


In [1]:
# Set up for running selenium in Google Colab
## You don't need to run this code if you do it in Jupyter notebook, or other local Python setting
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium



Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,552 kB]
Get:14 http://security.ubuntu.com

Install chromedriver-autoinstaller   


In [2]:
!pip install chromedriver-autoinstaller

Set up Selenium WebDriver with Chromedriver Auto-installation

In [3]:
# Import necessary modules from Selenium
from selenium import webdriver
import chromedriver_autoinstaller
def initialize_driver():
    # Configure Chrome options
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')  # Ensure GUI is off
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

    # Use chromedriver_autoinstaller to install the latest version of ChromeDriver
    chromedriver_autoinstaller.install()

    # Set up the WebDriver with configured options
    driver = webdriver.Chrome(options=chrome_options)

    return driver

# Import Necessary Libraries

In [4]:
# Import necessary modules from Selenium
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Import BeautifulSoup for HTML parsing
from bs4 import BeautifulSoup

# Import time for adding delays
import time

# Import csv for handling CSV files
import csv


# Function to Scrape Main Page: Extract Links from Table

In [5]:
def scrape_main_page(url, driver, table_xpath, data_list_name):
    """
    Scrape data from the given URL and table XPath and store the results in a list.

    :param url: The URL of the page to scrape.
    :param driver: The Selenium WebDriver instance.
    :param table_xpath: The XPath of the table to scrape.
    :param data_list_name: The name of the list to store extracted links.
    :return: A list of extracted links.
    """
    # Make an HTTP request to the main page
    driver.get(url)

    # Use WebDriverWait to wait for the "Show All" button to be clickable
    try:
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/div[2]/div[2]/div/div/div/div[3]/button'))
        )
        driver.execute_script("arguments[0].click();", button)
    except Exception as e:
        print("Error clicking the search button:", e)
        return []

    # Use WebDriverWait to wait for the table to be present
    try:
        table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, table_xpath))
        )
    except Exception as e:
        print("Error finding the table:", e)
        return []

    # Pause briefly to allow table data to load, especially if AJAX is involved
    time.sleep(2)

    data_list = []  # List to store extracted links

    # Extract the table rows using Selenium
    rows = table.find_elements(By.XPATH, './tbody/tr[@class="data-result results"]')

    if not rows:
        print("No rows found in the table. Check the HTML structure.")
    for row in rows:
        try:
            # Use Selenium to find the column and link
            country_td = row.find_element(By.CLASS_NAME, 'country')
            link_tag = country_td.find_element(By.TAG_NAME, 'a')
            link = link_tag.get_attribute('href')
            data_list.append(link)
        except Exception as e:
            print(f"Error extracting link from row: {e}")

    return data_list

# Function to Scrape connections details

In [6]:
def scrape_country_details(country_url, driver, rank, data_type):
    """
    Scrapes country details from the provided URL based on the data type (Broadband or Mobile).

    Parameters:
    - country_url: URL of the country details page.
    - driver: Selenium WebDriver instance.
    - rank: Rank of the country.
    - data_type: Type of data to scrape ('Broadband' or 'Mobile').

    Returns:
    - A dictionary with the country details.
    """
    # Make an HTTP request to the country details page
    driver.get(country_url)
    # Add a delay to ensure the page is fully loaded
    time.sleep(2)
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extract the country name from the page header
    page_header = soup.select_one('div.page-header-title')
    if page_header:
        full_text = page_header.get_text(separator=' ', strip=True)
        country_name = ' '.join(full_text.split()[:-4]).strip()
    else:
        country_name = 'Unknown'

    # Define selectors and parent container based on the data type
    if data_type == 'Broadband':
        container_id = 'column-fixedMean'
    elif data_type == 'Mobile':
        container_id = 'column-mobileMean'
    else:
        raise ValueError("Invalid data_type. Must be 'Broadband' or 'Mobile'.")

    # Locate the correct container
    container = soup.select_one(f'div#{container_id} .headings')

    # Extract data from the selected container
    if container:
        # Extract the download speed
        download_speed = container.select_one('div.result-group.result-group-icon.download span.number')
        download_speed = download_speed.text.strip() if download_speed else 'Unknown'

        # Extract the upload speed
        upload_speed = container.select_one('div.result-group.result-group-icon.upload span.number')
        upload_speed = upload_speed.text.strip() if upload_speed else 'Unknown'

        # Extract the latency
        latency = container.select_one('div.result-group.result-group-icon.latency span.number')
        latency = latency.text.strip() if latency else 'Unknown'
    else:
        download_speed = upload_speed = latency = 'Unknown'

    return {
        'rank': rank,
        'country': country_name,
        'upload_speed': upload_speed,
        'download_speed': download_speed,
        'latency': latency
    }


# Function to save data to a CSV File

In [7]:
def save_to_csv(data, file_name='data1.csv'):
    if not data:
        print("No data to save.")
        return

    # Assuming all dictionaries have the same keys
    headers = data[0].keys()

    # Open the CSV file in write mode, create a new file if it doesn't exist
    with open(file_name, 'w', newline='', encoding='utf-8') as csvfile:
        # Create a CSV writer with specified fieldnames
        writer = csv.DictWriter(csvfile, fieldnames=headers)

        # Write the headers
        writer.writeheader()

        # Write the data to the CSV file
        writer.writerows(data)

    print(f"Data saved to {file_name}.")


# Main function to execute the scraping process.


In [14]:
if __name__ == '__main__':

  # Initialize the Selenium WebDriver
  driver = initialize_driver()

  # URL of the page to scrape
  main_url = 'https://www.speedtest.net/global-index'

   # Scrape Fixed Broadband data
  data_Broadband = scrape_main_page(main_url, driver, '//*[@id="column-fixedMedian"]/div[1]/div/table', 'data_Broadband')
  # Scrape Mobile data
  data_Mobile = scrape_main_page(main_url, driver, '//*[@id="column-mobileMedian"]/div[1]/div/table', 'data_Mobile')

  # Scrape country details for Broadband and save to CSV
  broadband_data = [scrape_country_details(link, driver, rank,'Broadband') for rank, link in enumerate(data_Broadband, start=1)]
  save_to_csv(broadband_data, 'data_broadband.csv')

  # Scrape country details for Mobile and save to CSV
  mobile_data = [scrape_country_details(link, driver, rank,'Mobile') for rank, link in enumerate(data_Mobile, start=1)]
  save_to_csv(mobile_data, 'data_mobile.csv')

  # Quit the WebDriver session
  driver.quit()

  print("Data scraping process completed successfully.")

Data saved to data_broadband.csv.
Data saved to data_mobile.csv.
Data scraping process completed successfully.


# Converting csv to sql file

In [9]:
!pip install pymysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.6 MB/s eta 0:00:00


In [15]:
import pandas as pd
import pymysql

def escape_special_characters(value):
    """Escape special characters in a string for SQL insertion."""
    if isinstance(value, str):
        # Use pymysql's built-in method to escape string safely
        return pymysql.converters.escape_string(value)
    return value

def csv_to_sql(csv_file, table_name, sql_file):
    # Read the CSV file
    df = pd.read_csv(csv_file)

    # Open the SQL file with UTF-8 encoding
    with open(sql_file, 'w', encoding='utf-8') as f:
        # Write SQL insert statements
        for i, row in df.iterrows():
            try:
                # Escape special characters in the country field
                country = escape_special_characters(row['country'])
                rank = int(row['rank'])
                download_speed = float(row['download_speed']) if pd.notna(row['download_speed']) else None
                upload_speed = float(row['upload_speed']) if pd.notna(row['upload_speed']) else None
                latency = float(row['latency']) if pd.notna(row['latency']) else None

                # Construct the SQL insert statement
                sql = f"INSERT INTO {table_name} (country, rank, download, upload, latency) VALUES ("
                sql += f"'{country}', {rank}, {download_speed}, {upload_speed}, {latency});\n"

                # Write the SQL statement to the file
                f.write(sql)

            except ValueError as e:
                print(f"Error processing row {i}: {e}")
            except KeyError as e:
                print(f"Missing column in CSV file: {e}")

# Convert CSV files to SQL
csv_to_sql('data_mobile.csv', 'mobile', 'mobile_data.sql')
csv_to_sql('data_broadband.csv', 'broadband', 'broadband_data.sql')
